## Load PyEpiDAG

In [2]:
import epidag as dag
import numpy as np

## Define a DAG

Compose a script

```
PCore Exp1 {
    # Definitions of nodes
}
```

Then, .. parse to a DAG

In [3]:
script = '''
PCore Exp1 {
    n
    a = 0.5 
    b ~ beta(1, 1)
    c = min(a, b)
    y ~ binom(n, c)
    z = f(a,b)
}
'''

js = dag.bn_script_to_json(script)
bn = dag.BayesianNetwork(js)
bn


Name:	Exp1
Nodes:
	n
	b ~ beta(1,1)
	a = 0.5
	z = f(b, a)
	c = min(a,b)
	y ~ binom(n,c)

### Single value variable


> VariableName = Number 


In [4]:
SingleValue = bn['a']
print('Node \'a\'')
print('Definition:', SingleValue)

print('\nFind the value')
print(SingleValue())

Node 'a'
Definition: a = 0.5

Find the value
0.5


### Exogenous variable

> VariableName

In [5]:
Exogenous = bn['n']
print('Node \'n\'')
print('Definition:', Exogenous)

print('\nFind the value; must append external resources')
print(Exogenous({'n': 5}))

Node 'n'
Definition: n

Find the value; must append external resources
5


### Random variable

> VariableName ~ p(...)

** p(...) **: a probabilidy density/mass function

In [6]:
Random = bn['b']
print('Node \'b\'')
print('Definition:', Random)

print('\nSample a value')
print(Random())

Node 'b'
Definition: b ~ beta(1,1)

Sample a value
0.2841143537163034


### Equation output

> OutputName = g(...)

** g(...) **: a mathematical function

In [7]:
Equation = bn['c']
print('Node \'c\'')
print('Definition:', Equation)

parents = {
    'a': SingleValue(), 
    'b': Random() 
}

print('\nCalculate a value')
print(Equation(parents))

Node 'c'
Definition: c = min(a,b)

Calculate a value
0.4252355837097731


### Pseudo Node
Pseudo nodes are the nodes which can not be implemented in simulation but in relation inference.

> VariableName = f(...)

** f(...) **: a pseudo function start with ** f ** and a list of parent variables follows

In [8]:
Pseudo = bn['z']
print('Node \'z\'')
print('Definition:', Pseudo)

parents = {
    'a': SingleValue(), 
    'b': Random()
}
print('Parents:', Pseudo.Parents)

try:
    Pseudo(parents)
except AttributeError:
    print('Pseudo nodes do not provide any implementation')

Node 'z'
Definition: z = f(b, a)
Parents: {'b', 'a'}
Pseudo nodes do not provide any implementation


## For simulation model

The **'epidag.simulation'** is used to provide tools for simulation modelling. 

### Reasons to use 'epidag.simulation'
- The parameters of my model have complicated interactions among each other.
- My model includes many random effects, so I don't want to fix the parameters in the begining.
- I don't want to rebuild my model after the parameters changed. (Intervention analysis).
- My study include Monte Carlo inference and model fitting, so I need a convienant interface.


### SimulationCore
SimulationCore is a type of object carrying all the definition of a parameter model.


### ParameterCore
ParameterCore is a type of object can be use directly in a simulation model. A ParameterCore can be generated from its parent ParameterCore or a SimulationCore. After a ParameterCore instantiated 1) the fixed nodes are assigned, 2) the random nodes are ready to be used.

### Purposed workflow

Monte Carlo simulation
1. Prepare a SimulationCore
2. For each iteration, generate a ParameterCore
3. Build simulation models with the ParamterCores
4. Collect results and analysis (summarise or fit to data)

### Example 1. Basic syntex and function, Gaussian distribution

This example shows a normal variable ($x$) with a fixed mean value ($mu_x$) and a standard deviation ($sd$) distributed from an exponential distribution

#### Step 1. generate a simulation given a Bayesian network

In [9]:
script = '''
PCore Exp1 {
   mu_x = 0
   sd ~ exp(1)
   x ~ norm(mu_x, sd)
}
'''

bn = dag.bn_from_script(script)
sc = dag.as_simulation_core(bn)

#### Step 2. Instantiate a ParameterCore which 

- Hyper-parameter ($sd$) is fixed to a certain value
- A sampler of the leaf variable ($x$) is prepared

In [10]:
pc = sc.generate(nickname='exp2')
pc

sd: 0.953622, mu_x: 0

#### Step 3. Get sampler x from the ParameterCore and provide it to a simulation model.

- The sampler can be used repeatly
- You don't need to refer to its hyper-parameters while using

In [11]:
x = pc.get_sampler('x')
x

Actor x (norm(mu_x,sd)) on exp2

In [12]:
x(), np.mean([x() for _ in range(1000)])

(0.048272260597649976, -0.003228344679145397)

#### Step 4. Intervention

You can set impulse on the ParameterCore. Then, 
- The impulse will be passed down to its downstream variables
- You don't need to do anything to the sample  

In [13]:
pc.impulse({'sd': 5, 'mu_x': 100})
pc

sd: 5, mu_x: 100

In [14]:
x(), np.mean([x() for _ in range(1000)])

(101.24501872556885, 99.96975690031302)

### Example 2. Random effects, Beta-Binomial model

Example 2 is a standard example in Baysian inference. Beta-Binomial model are used to model count data ($x$) with a latent variable, probability ($prob$). 


> dag.as_simulation_core(bn, random=[...])

The option **random** defined variables which we do not want then be fixed during ParameterCore instantiation

In [15]:
script = '''
PCore BetaBinom {
    alpha = 1
    beta = 1
    n
    prob ~ beta(alpha, beta)
    x ~ binom(n, prob)
}
'''

bn = dag.bn_from_script(script)
sc = dag.as_simulation_core(bn, random=['prob'])

Since the variable $n$ is an exogenous variable, we introduce it to new ParameterCore by **exo={...}**.

To be noticed that, $prob$ has been set as a random effect, so the variable will be requested whenever new $x$ is requested

In [16]:
pc = sc.generate('exp1', exo={'n': 100})
pc

n: 100, beta: 1, alpha: 1

Again, we get a sampler $x$ for the generated ParameterCore

In [17]:
x = pc.get_sampler('x')
x()

17

**list_all** option print all variables used to sample outcome variable. You can see that $prob$ is not a fixed variable

### Example 3. Exposed variables,  Regression model

The example is a linear regression model. Dependant variable ($y$) is composed of $x*b1$ and an intercept $b0$; $var=eps^2$ is a measure of variance.

In [24]:
script = '''
PCore Regression {
    b0 = 1
    b1 = 0.5
    x = 1
    eps ~ norm(0, 1)
    y  = x*b1 + b0 + eps
    var = pow(eps, 2)
}
'''

bn = dag.bn_from_script(script)

However, $var$ is not a variable for simulation but for providing external information, so it do not need to be treated as a sampler.. 

In [25]:
sc = dag.as_simulation_core(bn)
pc = sc.generate('exp3-1')
print('Fixed variables', pc)
print('Samplers', pc.list_samplers())

Fixed variables eps: -0.973046, x: 1, b1: 0.5, b0: 1
Samplers ['y', 'var']


Use ** out ** option in generating ParameterCore can indicate the parameters should be exposed to a simulation model.

In [26]:
sc = dag.as_simulation_core(bn, out=['y'])
pc = sc.generate('exp3-2')
print('Fixed variables', pc)
print('Samplers', pc.list_samplers())

Fixed variables eps: -0.468521, var: 0.219512, x: 1, b1: 0.5, b0: 1
Samplers ['y']


### Example 4. Hierarchy, BMI model

Example 4 describes a parameter model of a BMI (body mass index) simulation model. The model include the layers of model: country, area, and people. A country have many areas; an area have many people; there are two types of people: agA and agB. 

- Each area have its amount of foodstores which can provide food to people.  
- agA and agB preform differently in the variance of BMI
- The simulation model requests the sex of agA individuals in order to model their behavour 

In [27]:
script = '''
PCore BMI {
    b0 ~ norm(12, 1)
    b1 = 0.5
    pf ~ beta(8, 20)
    foodstore ~ binom(100, pf)
    b0r ~ norm(0, .01)
    ageA ~ norm(20, 3)
    ageB ~ norm(30, 2)
    ps ~ beta(5, 6)
    sexA ~ cat({'m': ps, 'f': 1-ps})
    muA = b0 + b0r + b1*ageA
    bmiA ~ norm(muA, sd)
    sdB = sd * 0.5
    muB = b0 + b0r + b1*ageB
    bmiB ~ norm(muB, sdB)
}
'''

bn = dag.bn_from_script(script)

You can define hierarchies by a dictionary with 

1. parameter groups as keys and of respective parameters as values 
2. putting their children groups in the values as well

You do not need to list every variable. The variable outside the list will be optimalised in the SimulationCore

In [28]:
hie = {
    'country': ['area'],
    'area': ['b0r', 'pf', 'ps', 'foodstore', 'agA', 'agB'],
    'agA': ['bmiA', 'ageA', 'sexA'],
    'agB': ['bmiB', 'ageB']
}

sc = dag.as_simulation_core(bn, hie=hie,
                            random=['muA'],
                            out=['foodstore', 'bmiA', 'bmiB'])
sc.deep_print()

country(sd, b0, sdB, b1)
-- area(ps, foodstore, b0r, pf)
---- agB(ageB, bmiB, muB)
---- agA(sexA, muA, bmiA, ageA)


You can use a SimulationCore to generate a root ParameterCore and use the root to $breed$ children ParameterCores

In [29]:
pc = sc.generate('Taiwan', {'sd': 1})
pc_taipei = pc.breed('Taipei', 'area')
pc_taipei.breed('A1', 'agA')
pc_taipei.breed('A2', 'agA')
pc_taipei.breed('B1', 'agB')
pc_taipei.breed('B2', 'agB')

pc.deep_print()

Taiwan (sd: 1, sdB: 0.5, b1: 0.5, b0: 13.5058)
-- Taipei (ps: 0.432856, b0r: 0.00651805, pf: 0.213773)
---- A1 (sexA: m, ageA: 18.0488)
---- A2 (sexA: f, ageA: 26.7367)
---- B1 (ageB: 32.3215, muB: 29.673)
---- B2 (ageB: 29.2856, muB: 28.1551)


In [30]:
print('Get node from it parent')
a1 = pc_taipei.get_child('A1')
a1.print()

print('\nGet a node from root: use \'@\' to link names of ParameterCores')
a1 = pc.find_descendant('Taiwan@Taipei@A1')
a1.print()

Get node from it parent
A1 (sexA: m, ageA: 18.0488)

Get a node from root: use '@' to link names of ParameterCores
A1 (sexA: m, ageA: 18.0488)


When putting intervention in a node, the impulse will be passed to its children node automatically.

In [31]:
# see the change of muB
pc.impulse({'b0r': 10})
pc.deep_print()

Taiwan (sd: 1, sdB: 0.5, b1: 0.5, b0: 13.5058)
-- Taipei (ps: 0.432856, b0r: 10, pf: 0.213773)
---- A1 (sexA: m, ageA: 18.0488)
---- A2 (sexA: f, ageA: 26.7367)
---- B1 (ageB: 32.3215, muB: 39.6665)
---- B2 (ageB: 29.2856, muB: 38.1486)


In [32]:
pc.impulse(['b0r'])
pc.deep_print()

Taiwan (sd: 1, sdB: 0.5, b1: 0.5, b0: 13.5058)
-- Taipei (ps: 0.432856, b0r: -0.0253811, pf: 0.213773)
---- A1 (sexA: m, ageA: 18.0488)
---- A2 (sexA: f, ageA: 26.7367)
---- B1 (ageB: 32.3215, muB: 29.6411)
---- B2 (ageB: 29.2856, muB: 28.1232)


### Example 5. A simple agent-based model 

In [33]:
script = '''
PCore BetaBin {
    al = 1
    be = 1
    p ~ beta(al, be)
    x ~ binom(5, p)
}
'''


bn = dag.bn_from_script(script)

hie = {
    'root': ['ag'],
    'ag': ['x']
}

sm = dag.as_simulation_core(bn, hie)


In [34]:
class Agent:
    def __init__(self, name, p):
        self.Name = name
        self.Parameters = p
        self.X = p.get_sampler('x')
        
    def produce(self, k):
        return self.X.sample(k)

class AgentBasedModel:
    def __init__(self, pars, n_agents):
        self.Parameters = pars
        self.Agents = list()
        for i in range(n_agents):
            name = 'Ag{}'.format(i)
            p = pars.breed(name, 'ag')
            self.Agents.append(Agent(name, p))
        
    def product(self, k):
        return np.array([ag.produce(k) for ag in self.Agents]).sum()
        
        
        
def fn_sim(pars, data):
    abm = AgentBasedModel(pars, data['N'])
    return abm.product(data['K'])

def fn_mean(sim, data):
    return -abs(sim - data['X'])

In [35]:
pars = sm.generate('Simu A')

abm = AgentBasedModel(pars, 5)

In [36]:
abm.product(100)

2036

In [37]:
d = {
    'N': 10,
    'K': 10,
    'X': 200
}
pars = sm.generate('Simu A')
sim = fn_sim(pars, d)
mea = fn_mean(sim, d)
sim, mea

(216, -16)

In [38]:
sdm = dag.as_simulation_data_model(sm, d, fn_sim, fn_mean)

In [39]:
fit = dag.fitting.ABC(sdm)
fit.fit(100)
fit.report_posterior()

,al,be,p
count,100.0,100.0,100.000000
mean,1.0,1.0,0.400674
std,0.0,0.0,0.027777
min,1.0,1.0,0.333593
25%,1.0,1.0,0.381319
50%,1.0,1.0,0.402966
75%,1.0,1.0,0.420656
max,1.0,1.0,0.487924
